In [1]:
from langchain.vectorstores import Chroma
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.chains import VectorDBQA
import sentence_transformers
import pandas as pd

/Volumes/External/opt/anaconda3/envs/ollama2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%cd /Volumes/External/source/GTPracticum/data 
%ls -salh

/Volumes/External/source/GTPracticum/data
total 13162976
      0 drwxr-xr-x   26 kariato  staff   832B Mar 17 23:23 ./
      0 drwxr-xr-x    8 kariato  staff   256B Mar 15 14:14 ../
     16 -rw-r--r--@   1 kariato  staff   6.0K Mar 17 23:34 .DS_Store
  27280 -rw-r--r--@   1 kariato  staff    13M Feb 21 00:04 2023-10-eb-big-book-of-mlops-2nd-edition-v2-102723-final (1).pdf
  27280 -rw-r--r--@   1 kariato  staff    13M Feb 20 10:15 2023-10-eb-big-book-of-mlops-2nd-edition-v2-102723-final.pdf
    464 -rw-r--r--@   1 kariato  staff   232K Feb 10 16:00 9deec473-6ab3-4c90-a2ec-86ae93b0a6e6.pdf
   1472 -rw-r--r--@   1 kariato  staff   730K Mar 16 22:20 W-2_Form_2023_Davey_2024_01_05_13_13_13_-0800_W-2_ESS (1).pdf
   1472 -rw-r--r--@   1 kariato  staff   730K Mar 16 22:15 W-2_Form_2023_Davey_2024_01_05_13_13_13_-0800_W-2_ESS.pdf
   3104 -rw-r--r--@   1 kariato  staff   1.5M Mar 17 23:49 alltopics.csv
 733000 -rw-r--r--@   1 kariato  staff   358M Feb 10 14:56 archive (1).zip
      0 drwxr-xr-x 

/Volumes/External/opt/anaconda3/envs/ollama2/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
embedding_function2 = SentenceTransformerEmbeddings(model_name="all-mpnet-base-v2")

/Volumes/External/opt/anaconda3/envs/ollama2/lib/python3.11/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [4]:
db = Chroma(embedding_function=embedding_function,persist_directory="./chroma_db")
db2 = Chroma(embedding_function=embedding_function2,persist_directory="./chroma2_db")
db3 = Chroma(embedding_function=embedding_function2,persist_directory="./chroma3_db")
#qa = VectorDBQA.from_chain_type(llm=OpenAI(), chain_type="stuff", vectorstore=vectordb)

In [24]:
from langchain_community.llms import Ollama

llm = Ollama(model="jaskier",temperature=0)

#x=llm.invoke(prompt)
#print(x)

In [6]:
import re
def cleanText(text):
        
    text = text.replace('\\n','')
    text = text.replace('\\','')
    text = re.sub(' +', ' ', text)
    text = text.replace('\t', '')
    text = text.replace('\n', '')
    return text

In [32]:
import time
from collections import defaultdict
scores=defaultdict(dict)
#query = "it appears that some Enron employees used dummy accounts and rigged valuation methodologies to create false profit-and-loss entries for the derivatives"
query = "mark to market accounting practices"
print("Querying db")
docs = db.similarity_search_with_score(query,k=600)
print(len(docs))
topics={}
scr={}
for doc in docs:
    details = cleanText(doc[0].page_content)
    #print(doc[1])
    if details not in topics:
        topics[details]=doc[0].metadata
        topics[details]['score']=doc[1]
print(len(topics))
topicResults=[]
print("Querying llm")
print(len(scores))
i=0
models=[    'mistral',  'openhermes', 'vicuna', 'gemma', 'llama2-uncensored']
for model in models:
    #
    llm = Ollama(model=model,temperature=0)
    print(f"Querying llm {model}")
    for email in list(topics.keys()):
        if email not in scores or model not in scores[email]:
        
            start_time = time.time()
            x=llm.invoke(f"""You are a finance specialist does the following email try to hide loses: These are the enron emails. They used SBE with the name of Raptors to hide loses using mark to market accounting so revenue was realized as profit. Do any of the following emails indicate an abuse of the accounting rules. Please respond to each email with a number on the scale of 0 to 10. where 0 is no abuse while 10 is outright fraud.
Example of such a email:
<Email Start>I'm concerned that the mark to market accounting practices that we placed the loses in the Raptor SBE could be viewed as Fraud
<Email End>
<Score>10<End Score>
<Email Start>Accounting practices  did not report losses so revenue was realized as profit hidding true financial position
<Email End>
<Score>10<End Score>                 
<Email Start>Accounting practices  we placed on the books are 100% legal and trasparent
<Email End>
<Score>0<End Score>
<Email Start>News about the accounting practices at Enron discuss devrivatives and mark to market accounting
<Email End>
<Score>0<End Score>                     
Please score the following email:
<Email Start>
{email}
<Email End>
""")
            scores[email][model]=x
            i+=1
            print("--- %s seconds ---" % (time.time() - start_time),i)
        else:
            i+=1
            print(f"Already scored {i}")
        #print(x)
results=[]
model='all'
for idx,j in scores.items():
    j['email']=idx
    j.update(topics[idx])
    results.append(j)
resultsDF=pd.DataFrame(results)
resultsDF.to_csv(f"resultsMix2_{model}.csv")

Querying db
600
256
Querying llm
0
Querying llm mistral
--- 18.146378993988037 seconds --- 1
--- 3.9028940200805664 seconds --- 2
--- 3.522249937057495 seconds --- 3
--- 4.473380088806152 seconds --- 4
--- 7.698180198669434 seconds --- 5
--- 6.41375207901001 seconds --- 6
--- 6.836760997772217 seconds --- 7
--- 4.954282999038696 seconds --- 8
--- 7.413327693939209 seconds --- 9
--- 7.776710033416748 seconds --- 10
--- 7.828747034072876 seconds --- 11
--- 12.7291579246521 seconds --- 12
--- 4.249345064163208 seconds --- 13
--- 9.960734844207764 seconds --- 14
--- 4.316695213317871 seconds --- 15
--- 11.106553077697754 seconds --- 16
--- 4.659502983093262 seconds --- 17
--- 9.56882095336914 seconds --- 18
--- 12.109672784805298 seconds --- 19
--- 4.797725200653076 seconds --- 20
--- 3.3922412395477295 seconds --- 21
--- 2.8195438385009766 seconds --- 22
--- 10.160510063171387 seconds --- 23
--- 10.024769067764282 seconds --- 24
--- 6.506580829620361 seconds --- 25
--- 8.296093940734863 s

In [29]:
results=[]
for i,j in scores.items():
    j['email']=i
    j.update(topics[i])
    results.append(j)
resultsDF=pd.DataFrame(results)
resultsDF.to_csv("resultsMix3.csv")


In [31]:
resultsDF.columns


Index(['llama2-uncensored', 'email', 'DATE', 'SUBJECT', 'Unnamed: 0', 'score',
       'FROM', 'TO', 'SENT', 'CC'],
      dtype='object')

In [33]:
resultsDF.head().T

,0,1,2,3,4
mistral,<Score>0<End Score>\n\nThe email does not con...,<Score>0<End Score>\n\nThe email does not con...,<Score>0<End Score>\n\nThe email itself does ...,<Score>0<End Score>\n\nThe email does not con...,<Score>0<End Score>\n\nThe email does not con...
openhermes,<Score>0<End Score>,<Score>0<End Score>,<Score>0<End Score>,<Score>2<End Score>,<Score>0<End Score>
vicuna,<Score>8<End Score>,<Email Start>\nSubject: Daily Position Report ...,<Email Start>\nSubject: Re: WSJ Article Mark t...,<Email Start>Subject: WSJ Article on Enron's M...,<Score>7<End Score>
gemma,## Email Score\n\n**Email Start:**\n\nSubject:...,## Email Score\n\n**Email Start:**\n\nSubject:...,## Email Score\n\n**Email Start:**\n\nSubject:...,## Email Score\n\n**Email Start:**\n\nSubject:...,## Score for Email\n\n**Score:** 2\n\n**Explan...
llama2-uncensored,The email that discusses the news about Enron'...,The email with a score of 10 indicates an outr...,The first email is a 10/10 for abuse of accoun...,The email that discusses the mark to market ac...,"The email with the subject ""Roger Ondreko's Pr..."
email,Subject: Re: WSJ Article on Enron's Mark to Ma...,Subject: Daily Position Report class - sounds ...,Subject: WSJ Article Mark to Market MemoPlease...,Subject: WSJ Article on Enron's Mark to Market...,Subject: Roger Ondreko's Presentation on Marke...
DATE,"Fri, 22 Sep 2000 15:37","Wed, 23 Feb 2000 01:44","Wed, 20 Sep 2000 02:26","Fri, 22 Sep 2000 10:33","Tue, 24 Oct 2000 05:25"
SUBJECT,Re,>>,NaN,NaN,>>
Unnamed: 0,77277,384559,227836,59549,301715
score,0.562741,0.703303,0.758774,0.771171,0.780912


In [35]:
import time
from collections import defaultdict
scores=defaultdict(dict)
#query = "it appears that some Enron employees used dummy accounts and rigged valuation methodologies to create false profit-and-loss entries for the derivatives"
query =  "raptor creation of special purpose entities"
print("Querying db")
docs = db.similarity_search_with_score(query,k=600)
print(len(docs))
topics={}
scr={}
for doc in docs:
    details = cleanText(doc[0].page_content)
    #print(doc[1])
    if details not in topics:
        topics[details]=doc[0].metadata
        topics[details]['score']=doc[1]
print(len(topics))
topicResults=[]
print("Querying llm")
print(len(scores))
i=0
models=[    'mistral',  'openhermes', 'vicuna', 'gemma', 'llama2-uncensored']
for model in models:
    #
    llm = Ollama(model=model,temperature=0)
    print(f"Querying llm {model}")
    for email in list(topics.keys()):
        if email not in scores or model not in scores[email]:
        
            start_time = time.time()
            x=llm.invoke(f"""You are a finance specialist does the following email try to hide loses: These are the enron emails. They used SBE with the name of Raptors to hide loses using mark to market accounting so revenue was realized as profit. Do any of the following emails indicate an abuse of the accounting rules. Please respond to each email with a number on the scale of 0 to 10. where 0 is no abuse while 10 is outright fraud.
Example of such a email:
<Email Start>I'm concerned that the mark to market accounting practices that we placed the loses in the Raptor SBE could be viewed as Fraud
<Email End>
<Score>10<End Score>
<Email Start>Accounting practices  did not report losses so revenue was realized as profit hidding true financial position
<Email End>
<Score>10<End Score>                 
<Email Start>Accounting practices  we placed on the books are 100% legal and trasparent
<Email End>
<Score>0<End Score>
<Email Start>News about the accounting practices at Enron discuss devrivatives and mark to market accounting
<Email End>
<Score>0<End Score>                     
Please score the following email:
<Email Start>
{email}
<Email End>
""")
            scores[email][model]=x
            i+=1
            print("--- %s seconds ---" % (time.time() - start_time),i)
        else:
            i+=1
            print(f"Already scored {i}")
        #print(x)
results=[]
model='all'
for idx,j in scores.items():
    j['email']=idx
    j.update(topics[idx])
    results.append(j)
resultsDF=pd.DataFrame(results)
resultsDF.to_csv(f"resultsMix3_{model}.csv")

Querying db
600
290
Querying llm
0
Querying llm mistral
--- 17.531227827072144 seconds --- 1
--- 4.557079076766968 seconds --- 2
--- 3.0837929248809814 seconds --- 3
--- 3.118680000305176 seconds --- 4
--- 5.81890082359314 seconds --- 5
--- 3.908447027206421 seconds --- 6
--- 4.15209698677063 seconds --- 7
--- 4.503992795944214 seconds --- 8
--- 3.660292863845825 seconds --- 9
--- 5.819736957550049 seconds --- 10
--- 3.52354097366333 seconds --- 11
--- 3.8482720851898193 seconds --- 12
--- 2.6564719676971436 seconds --- 13
--- 2.655046224594116 seconds --- 14
--- 4.0458948612213135 seconds --- 15
--- 7.649841785430908 seconds --- 16
--- 2.274600028991699 seconds --- 17
--- 3.857692241668701 seconds --- 18
--- 3.540065050125122 seconds --- 19
--- 2.9082581996917725 seconds --- 20
--- 4.050575256347656 seconds --- 21
--- 3.956775188446045 seconds --- 22
--- 4.052804231643677 seconds --- 23
--- 4.076849937438965 seconds --- 24
--- 4.99104905128479 seconds --- 25
--- 5.135876893997192 seco

In [41]:
import time
from collections import defaultdict
#scores=defaultdict(dict)
skip=1
query = "it appears that some Enron employees used dummy accounts and rigged valuation methodologies to create false profit-and-loss entries for the derivatives"
#query =  "raptor creation of special purpose entities"
print("Querying db")
docs = db.similarity_search_with_score(query,k=600)
print(len(docs))
topics={}
scr={}
for doc in docs:
    details = cleanText(doc[0].page_content)
    #print(doc[1])
    if details not in topics:
        topics[details]=doc[0].metadata
        topics[details]['score']=doc[1]
print(len(topics))
topicResults=[]
print("Querying llm")
print(len(scores))
i=0
models=[    'mistral',  'openhermes', 'vicuna', 'gemma', 'llama2-uncensored']
for model in models:
    #
    llm = Ollama(model=model,temperature=0)
    print(f"Querying llm {model}")
    for email in list(topics.keys()):
        if (email not in scores or model not in scores[email]) :
            if skip==0:
                start_time = time.time()
                x=llm.invoke(f"""You are a finance specialist does the following email try to hide loses: These are the enron emails. They used SBE with the name of Raptors to hide loses using mark to market accounting so revenue was realized as profit. Do any of the following emails indicate an abuse of the accounting rules. Please respond to each email with a number on the scale of 0 to 10. where 0 is no abuse while 10 is outright fraud.
    Example of such a email:
    <Email Start>I'm concerned that the mark to market accounting practices that we placed the loses in the Raptor SBE could be viewed as Fraud
    <Email End>
    <Score>10<End Score>
    <Email Start>Accounting practices  did not report losses so revenue was realized as profit hidding true financial position
    <Email End>
    <Score>10<End Score>                 
    <Email Start>Accounting practices  we placed on the books are 100% legal and trasparent
    <Email End>
    <Score>0<End Score>
    <Email Start>News about the accounting practices at Enron discuss devrivatives and mark to market accounting
    <Email End>
    <Score>0<End Score>                     
    Please score the following email:
    <Email Start>
    {email}
    <Email End>
    """)
                scores[email][model]=x
                i+=1
                print("--- %s seconds ---" % (time.time() - start_time),i)
            else:
                skip-=1
                scores[email][model]='hang'
                print(f"Skipping {email} {model} {skip}")
        else:
            i+=1
            print(f"Already scored {i}")
        #print(x)
results=[]
model='all'
for idx,j in scores.items():
    j['email']=idx
    j.update(topics[idx])
    results.append(j)
resultsDF=pd.DataFrame(results)
resultsDF.to_csv(f"resultsMix4_{model}.csv")

Querying db
600
342
Querying llm
342
Querying llm mistral
Already scored 1
Already scored 2
Already scored 3
Already scored 4
Already scored 5
Already scored 6
Already scored 7
Already scored 8
Already scored 9
Already scored 10
Already scored 11
Already scored 12
Already scored 13
Already scored 14
Already scored 15
Already scored 16
Already scored 17
Already scored 18
Already scored 19
Already scored 20
Already scored 21
Already scored 22
Already scored 23
Already scored 24
Already scored 25
Already scored 26
Already scored 27
Already scored 28
Already scored 29
Already scored 30
Already scored 31
Already scored 32
Already scored 33
Already scored 34
Already scored 35
Already scored 36
Already scored 37
Already scored 38
Already scored 39
Already scored 40
Already scored 41
Already scored 42
Already scored 43
Already scored 44
Already scored 45
Already scored 46
Already scored 47
Already scored 48
Already scored 49
Already scored 50
Already scored 51
Already scored 52
Already scored 

In [42]:
results=[]
model='all'
for idx,j in scores.items():
    j['email']=idx
    j.update(topics[idx])
    results.append(j)
resultsDF=pd.DataFrame(results)
resultsDF.to_csv(f"resultsMix4a_{model}.csv")
resultsDF.shape

(342, 14)

In [44]:
scores=defaultdict(dict)
skip=0

In [45]:
import time
from collections import defaultdict
#scores=defaultdict(dict)
#query = "it appears that some Enron employees used dummy accounts and rigged valuation methodologies to create false profit-and-loss entries for the derivatives"
query =  "record keeping accounting practices"
print("Querying db")
docs = db.similarity_search_with_score(query,k=600)
print(len(docs))
topics={}
scr={}
for doc in docs:
    details = cleanText(doc[0].page_content)
    #print(doc[1])
    if details not in topics:
        topics[details]=doc[0].metadata
        topics[details]['score']=doc[1]
print(len(topics))
topicResults=[]
print("Querying llm")
print(len(scores))
i=0
models=[    'mistral',  'openhermes', 'vicuna', 'gemma', 'llama2-uncensored']
for model in models:
    #
    llm = Ollama(model=model,temperature=0)
    print(f"Querying llm {model}")
    for email in list(topics.keys()):
        if (email not in scores or model not in scores[email]) :
            if skip==0:
                start_time = time.time()
                x=llm.invoke(f"""You are a finance specialist does the following email try to hide loses: These are the enron emails. They used SBE with the name of Raptors to hide loses using mark to market accounting so revenue was realized as profit. Do any of the following emails indicate an abuse of the accounting rules. Please respond to each email with a number on the scale of 0 to 10. where 0 is no abuse while 10 is outright fraud.
    Example of such a email:
    <Email Start>I'm concerned that the mark to market accounting practices that we placed the loses in the Raptor SBE could be viewed as Fraud
    <Email End>
    <Score>10<End Score>
    <Email Start>Accounting practices  did not report losses so revenue was realized as profit hidding true financial position
    <Email End>
    <Score>10<End Score>                 
    <Email Start>Accounting practices  we placed on the books are 100% legal and trasparent
    <Email End>
    <Score>0<End Score>
    <Email Start>News about the accounting practices at Enron discuss devrivatives and mark to market accounting
    <Email End>
    <Score>0<End Score>                     
    Please score the following email:
    <Email Start>
    {email}
    <Email End>
    """)
                scores[email][model]=x
                i+=1
                print("--- %s seconds ---" % (time.time() - start_time),i)
            else:
                skip-=1
                scores[email][model]='hang'
                print(f"Skipping {email} {model} {skip}")
        else:
            i+=1
            print(f"Already scored {i}")
        #print(x)
        #print(x)
results=[]
model='all'
for idx,j in scores.items():
    j['email']=idx
    j.update(topics[idx])
    results.append(j)
resultsDF=pd.DataFrame(results)
resultsDF.to_csv(f"resultsMix6_{model}.csv")

Querying db
600
307
Querying llm
0
Querying llm mistral
--- 8.236916065216064 seconds --- 1
--- 6.371081113815308 seconds --- 2
--- 10.25141716003418 seconds --- 3
--- 5.6597819328308105 seconds --- 4
--- 8.338778972625732 seconds --- 5
--- 10.389668941497803 seconds --- 6
--- 3.352882146835327 seconds --- 7
--- 7.822736740112305 seconds --- 8
--- 8.365654945373535 seconds --- 9
--- 7.460535049438477 seconds --- 10
--- 14.644362926483154 seconds --- 11
--- 7.490654945373535 seconds --- 12
--- 3.614694833755493 seconds --- 13
--- 15.754616022109985 seconds --- 14
--- 8.373269081115723 seconds --- 15
--- 5.254549980163574 seconds --- 16
--- 6.979875087738037 seconds --- 17
--- 3.751911163330078 seconds --- 18
--- 5.771327018737793 seconds --- 19
--- 8.62169599533081 seconds --- 20
--- 6.756669044494629 seconds --- 21
--- 3.910100221633911 seconds --- 22
--- 6.971081256866455 seconds --- 23
--- 8.2864511013031 seconds --- 24
--- 8.029568910598755 seconds --- 25
--- 4.902186155319214 secon

In [46]:
scores=defaultdict(dict)
skip=0

In [47]:
import time
query = "mark to market losses"
docs = db.similarity_search_with_score(query,k=600)
print(len(docs))
topics={}
scr={}
for doc in docs:
    details = cleanText(doc[0].page_content)
    #print(doc[1])
    if details not in topics:
        topics[details]=doc[0].metadata
        topics[details]['score']=doc[1]
print(len(topics))
topicResults=[]
print("Querying llm")
print(len(scores))
i=0
models=[    'mistral',  'openhermes', 'vicuna', 'gemma', 'llama2-uncensored']
for model in models:
    #
    llm = Ollama(model=model,temperature=0)
    print(f"Querying llm {model}")
    for email in list(topics.keys()):
        if (email not in scores or model not in scores[email]) :
            if skip==0:
                start_time = time.time()
                x=llm.invoke(f"""You are a finance specialist does the following email try to hide loses: These are the enron emails. They used SBE with the name of Raptors to hide loses using mark to market accounting so revenue was realized as profit. Do any of the following emails indicate an abuse of the accounting rules. Please respond to each email with a number on the scale of 0 to 10. where 0 is no abuse while 10 is outright fraud.
    Example of such a email:
    <Email Start>I'm concerned that the mark to market accounting practices that we placed the loses in the Raptor SBE could be viewed as Fraud
    <Email End>
    <Score>10<End Score>
    <Email Start>Accounting practices  did not report losses so revenue was realized as profit hidding true financial position
    <Email End>
    <Score>10<End Score>                 
    <Email Start>Accounting practices  we placed on the books are 100% legal and trasparent
    <Email End>
    <Score>0<End Score>
    <Email Start>News about the accounting practices at Enron discuss devrivatives and mark to market accounting
    <Email End>
    <Score>0<End Score>                     
    Please score the following email:
    <Email Start>
    {email}
    <Email End>
    """)
                scores[email][model]=x
                i+=1
                print("--- %s seconds ---" % (time.time() - start_time),i)
            else:
                skip-=1
                scores[email][model]='hang'
                print(f"Skipping {email} {model} {skip}")
        else:
            i+=1
            print(f"Already scored {i}")
        #print(x)
        #print(x)
results=[]
model='all'
for idx,j in scores.items():
    j['email']=idx
    j.update(topics[idx])
    results.append(j)
resultsDF=pd.DataFrame(results)
resultsDF.to_csv(f"resultsMix8_{model}.csv")

600
277
Querying llm
0
Querying llm mistral
--- 19.91873002052307 seconds --- 1
--- 4.557523965835571 seconds --- 2
--- 3.440493106842041 seconds --- 3
--- 2.923313856124878 seconds --- 4


In [ ]:
import time
from collections import defaultdict
scores=defaultdict(dict)
#query = "it appears that some Enron employees used dummy accounts and rigged valuation methodologies to create false profit-and-loss entries for the derivatives"
query = "mark to market accounting practices"
print("Querying db")
docs = db.similarity_search_with_score(query,k=600)
print(len(docs))
topics={}
scr={}
for doc in docs:
    details = cleanText(doc[0].page_content)
    #print(doc[1])
    if details not in topics:
        topics[details]=doc[0].metadata
        topics[details]['score']=doc[1]
print(len(topics))
topicResults=[]
print("Querying llm")
print(len(scores))
i=0
models=[    'mistral',  'openhermes', 'vicuna', 'gemma', 'llama2-uncensored']
for model in models:
    #
    llm = Ollama(model=model,temperature=0)
    print(f"Querying llm {model}")
    for email in list(topics.keys()):
        if email not in scores or model not in scores[email]:
        
            start_time = time.time()
            x=llm.invoke(f"""You are a finance specialist does the following email try to hide loses: These are the enron emails. They used SBE with the name of Raptors to hide loses using mark to market accounting so revenue was realized as profit. Do any of the following emails indicate an abuse of the accounting rules. Please respond to each email with a number on the scale of 0 to 10. where 0 is no abuse while 10 is outright fraud.
Example of such a email:
<Email Start>I'm concerned that the mark to market accounting practices that we placed the loses in the Raptor SBE could be viewed as Fraud
<Email End>
<Score>10<End Score>
<Email Start>Accounting practices  did not report losses so revenue was realized as profit hidding true financial position
<Email End>
<Score>10<End Score>                 
<Email Start>Accounting practices  we placed on the books are 100% legal and trasparent
<Email End>
<Score>0<End Score>
<Email Start>News about the accounting practices at Enron discuss devrivatives and mark to market accounting
<Email End>
<Score>0<End Score>                     
Please score the following email:
<Email Start>
{email}
<Email End>
""")
            scores[email][model]=x
            i+=1
            print("--- %s seconds ---" % (time.time() - start_time),i)
        else:
            i+=1
            print(f"Already scored {i}")
        #print(x)
results=[]
model='all'
for idx,j in scores.items():
    j['email']=idx
    j.update(topics[idx])
    results.append(j)
resultsDF=pd.DataFrame(results)
resultsDF.to_csv(f"resultsMix2_{model}.csv")

Querying db
600
256
Querying llm
0
Querying llm mistral
--- 18.146378993988037 seconds --- 1
--- 3.9028940200805664 seconds --- 2
--- 3.522249937057495 seconds --- 3
--- 4.473380088806152 seconds --- 4
--- 7.698180198669434 seconds --- 5
--- 6.41375207901001 seconds --- 6
--- 6.836760997772217 seconds --- 7
--- 4.954282999038696 seconds --- 8
--- 7.413327693939209 seconds --- 9
--- 7.776710033416748 seconds --- 10
--- 7.828747034072876 seconds --- 11
--- 12.7291579246521 seconds --- 12
--- 4.249345064163208 seconds --- 13
--- 9.960734844207764 seconds --- 14
--- 4.316695213317871 seconds --- 15
--- 11.106553077697754 seconds --- 16
--- 4.659502983093262 seconds --- 17
--- 9.56882095336914 seconds --- 18
--- 12.109672784805298 seconds --- 19
--- 4.797725200653076 seconds --- 20
--- 3.3922412395477295 seconds --- 21
--- 2.8195438385009766 seconds --- 22
--- 10.160510063171387 seconds --- 23
--- 10.024769067764282 seconds --- 24
--- 6.506580829620361 seconds --- 25
--- 8.296093940734863 s

In [73]:
import time
query = "raptor creation of special purpose entities"
docs = db2.similarity_search_with_score(query,k=2000)
topics=[]
for doc in docs:
    details = cleanText(doc[0].page_content)
    #print(doc[1])
    if details not in topics:
        topics.append(details)
topicResults=[]
i=0
print("sents",len(topics))
for email in topics:
    start_time = time.time()
    x=llm.invoke(f"""You are a finance specialist does the following email try to hide loses: These are the enron emails. They used SBE with the name of Raptors to hide loses using mark to market accounting so revenue was realized as profit. Do any of the following emails indicate an abuse of the accounting rules. Please respond to each email with a number on the scale of 0 to 10. where 0 is no abuse while 10 is outright fraud.
Example of such a email:
<Email Start>I'm concerned that the mark to market accounting practices that we placed the loses in the Raptor SBE could be viewed as Fraud
<Email End>
<Score>10<End Score>
<Email Start>Accounting practices  did nit report losses so revenue was realized as profit hidding true financial position
<Email End>
<Score>10<End Score>                 
<Email Start>Accounting practices  we placed on the books are 100% legal and trasparent
<Email End>
<Score>0<End Score>
Please score the following email:
<Email Start>
{email}
<Email End>
""")
    topicResults.append({'email':email,'score':x})
    i+=1
    print("--- %s seconds ---" % (time.time() - start_time),i)
topicResultsDF=pd.DataFrame(topicResults)
topicResultsDF.to_csv("topicResults2a.csv")

sents 474
--- 7.339417934417725 seconds --- 1
--- 1.4742741584777832 seconds --- 2
--- 3.5416150093078613 seconds --- 3
--- 2.852958917617798 seconds --- 4
--- 0.8699069023132324 seconds --- 5
--- 7.15320897102356 seconds --- 6
--- 4.42262601852417 seconds --- 7
--- 9.026015996932983 seconds --- 8
--- 1.3027970790863037 seconds --- 9
--- 2.153749942779541 seconds --- 10
--- 6.821197032928467 seconds --- 11
--- 10.051400899887085 seconds --- 12
--- 7.092861890792847 seconds --- 13
--- 3.1358163356781006 seconds --- 14
--- 6.232222080230713 seconds --- 15
--- 1.5600910186767578 seconds --- 16
--- 1.9166688919067383 seconds --- 17
--- 1.0818991661071777 seconds --- 18
--- 11.035797119140625 seconds --- 19
--- 2.841470956802368 seconds --- 20
--- 1.297400951385498 seconds --- 21
--- 6.729027986526489 seconds --- 22
--- 1.0856177806854248 seconds --- 23
--- 8.554369926452637 seconds --- 24
--- 1.4863121509552002 seconds --- 25
--- 4.540236949920654 seconds --- 26
--- 6.333446025848389 secon

In [64]:
topicResultsDF.head()

,email,score
0,"linked to its collapse. On the outside, they w...",<Score>8<End Score>
1,MANIPULATION ALLEGED But Enron's derivatives p...,<Score>8<End Score>
2,After listening to testimony at his committee'...,<Score>8<End Score>
3,"its derivatives trading revenue. ""In a nutshel...",\n<Score>7<End Score>
4,"Inside the web of this transaction, hundreds o...",<Score>8<End Score>


In [66]:
topicResultsDF[topicResultsDF['score'].str.contains('<Score>10')]['email'].to_list()

['********************************************************************** This e-mail is the property of Enron Corp. and/or its relevant affiliate and may contain confidential and privileged material for the sole',
 "Subject: Enron's Texas Style TradingYou have two cows. You sell three of them to your publically listed company using letters of credit opened by your brother-in-law at the bank. Then, you execute a debt-equity swap with an associated general officer, so you get all four cows back, with a tax-exemption for the fifth cow, of course.The rights to the estimated future value of the milk cash flow stream of the six cows are monetized and transferred via an intermediary to the Caymen Islands, secretly owned by a majority shareholder who sells the rights to all seven cows back to your listed company, off-balance sheet, of course. The annual report says the company owns eight cows and recognizes milk revenue for the 20 year life of the cows.This is hedged with derivative cow life i

In [67]:
topicResults

[{'email': "linked to its collapse. On the outside, they were used to create thecompany's web of off-balance sheet deals with complex financialpartnerships known as special-purpose vehicles. After some of those dealswent sour, Enron in October took a $1 billion charge against earnings anda $1.2 billion write-down in shareholder equity, triggering its nosediveinto bankruptcy. But Enron's derivatives problems ran far deeper than theoutside special purpose vehicles. It appears that some Enron employeesused dummy accounts and rigged valuation methodologies to create falseprofit and loss entries for the derivatives Enron traded.-------------------------------------------------------------",
  'score': '<Score>8<End Score>'},
 {'email': 'MANIPULATION ALLEGED But Enron\'s derivatives problems ran far deeper than the outside special purpose vehicles, Partnoy said, adding he had also gathered information indicating widespread manipulation of the company\'s derivatives trading revenues. "In a nu

In [28]:
query = "accounting aggresive hiding loses"
docs = db2.similarity_search_with_score(query,k=100)
for doc in docs:
    print(doc[0].page_content)

Subject: 

T
he first detailed allegations that Enron
abused mark-to-market accounting to
hide wholesale trading losses have
emerged in congressional testimony pro-vided
by a professor and former deriva-tives
trader who reviewed the Enron case
for Senate Governmental Affairs Com-mittee.
Those charges, along with allega-tions
that have surfaced that mark-to-market
accounting was misused by Enron
Energy Services, the company's retail
energy and services unit, are expected to
support the push by accounting rulemak-ers
for much more disclosure on mark-to-market
accounting practices and the
impact of mark-to-market accounting on
earnings trading firms' earnings.
In testimony Thursday before the
Senate Governmental Affairs Committee
last week, Frank Partnoy of the Universi-ty
of San Diego Law School, said it
appears that Enron traders mismarked
forward curves and manipulated the
amount of profits and losses that would
be reported in financial statements.
"In a nutshell, it appears that some


In [8]:
from collections import defaultdict
cts=defaultdict(list)

query = "mark to market"
docs = db2.similarity_search_with_score(query,k=100)
for doc in docs:
    cts[doc[0].page_content].append(doc[0].metadata['Unnamed: 0'])
cts

defaultdict(list,
            {'Subject: Mark to market\n\nSally, Could you please give me a call regarding our discussion about the above. thanks,Fred': [25342,
              25342],
             'Subject: WSJ Article Mark to Market Memo\n\nPlease see the revised memo from Steve Kean regarding WSJ article on Mark to \nMarket Accounting.': [227836,
              236366,
              236791,
              239235,
              244698,
              252107,
              227836,
              236366,
              236791,
              239235,
              244698,
              252107],
             'Subject: Daily Position Report class - sounds good; go ahead and sign up\n\n---------------------- Forwarded by Elizabeth Sager/HOU/ECT on 02/23/2000 \n   \n\tEnron North America Corp.\n\t\n\t\n\nSubject: Daily Position Report class offer\n\nDaily Position Report - February 25, 10:00 am - 2:00pm in EB46C1\nPresenters:  Georganne Hodges\n           Mike Moscoso\n\n\nThis course provides an 

In [ ]:
[227836,236366,236791,239235,244698,252107,227836,236366,236791,239235,244698,252107]

In [33]:
query = "california market manipulation"
docs = db2.similarity_search_with_score(query,k=100)
for doc in docs:
    print(doc[0].page_content)

Subject: FERC Staff market mitigation comments for California--Comments of
 Enron--DUE WEDNESDAY

I forgot to copy you ...
---------------------- Forwarded by Alan Comnes/PDX/ECT on 03/20/2001 10:00 
PM ---------------------------


Alan Comnes
03/20/2001 09:54 PM
Richter/HOU/ECT@ECT, Robert Badeer/HOU/ECT@ECT, Steve C Hall/PDX/ECT@ECT, 
Bill Williams III/PDX/ECT@ECT

Subject: FERC Staff market mitigation comments for California--Comments of 
Enron--DUE WEDNESDAY

To West Desk Traders in Cali:

On Thursday 3/22, FERC will accept comments on its staff's plan to provide 
market mitigation to California ISO markets.  A summary of the report is 
copied below.

Enron GA plans to file a motion that will support the comments of EPSA and 
WPTF and supplement the comments as noted below. 

EPSA's comments are generally supportive of the staff report but recommends 
that FERC:

clarify that staff's market mitigation measures are being put into place to 
address structural defects in the Californ

In [25]:
#  'similarity_search',
#  'similarity_search_by_vector',
#  'similarity_search_by_vector_with_relevance_scores',
#  'similarity_search_with_relevance_scores',
#  'similarity_search_with_score'
query = "mark to market losses"
docs = db3.similarity_search_with_relevance_scores(query)
print(docs[0][0].page_content)

Subject: Debt Trading Losses

We have discovered two positions in the Debt Trading book which were not 
being marked to market. The current mark of the 2 positions results in 
approximately $4.7 million loss.  The details are in :



The primary control issues are:
? Trader did not adjust price to reflect the effect of market movements
? Operations did not perform monthly price verification processes
? Desk management failed to recognize that the marks on these two trades had 
not moved over the term of the transactions (Q1 '00 - Nov 15th)

The losses have been recognized and a full review of the book is ongoing by 
Operations.  The trader has been relieved of his responsibilities and we will 
implement the steps agreed with Operations to ensure that we have a robust 
approach to controls for the Debt Trading book going forward.
